In [1]:
push!( LOAD_PATH, "." ); # folder que continen los modulos. Correr desde ahí dentro.
#
using Z_auxiliaresBRW, Z_auxiliaresGEN
using HDF5

In [2]:
arxname="/home/karel/BRWFiles/Isabel2019/control_02.brw"

"/home/karel/BRWFiles/Isabel2019/control_02.brw"

In [3]:
pmem=filesize(arxname)/Sys.total_memory()

0.5216578024702109

In [53]:
?in

search: in inv In Int Inf Int8 invoke Int64 Int32 Int16 invmod Inf64 Inf32 Inf16



```
in(x)
```

Create a function that checks whether its argument is [`in`](@ref) `x`, i.e. a function equivalent to `y -> y in x`.

The returned function is of type `Base.Fix2{typeof(in)}`, which can be used to implement specialized methods.

---

```
in(item, collection) -> Bool
∈(item, collection) -> Bool
∋(collection, item) -> Bool
```

Determine whether an item is in the given collection, in the sense that it is [`==`](@ref) to one of the values generated by iterating over the collection. Returns a `Bool` value, except if `item` is [`missing`](@ref) or `collection` contains `missing` but not `item`, in which case `missing` is returned ([three-valued logic](https://en.wikipedia.org/wiki/Three-valued_logic), matching the behavior of [`any`](@ref) and [`==`](@ref)).

Some collections follow a slightly different definition. For example, [`Set`](@ref)s check whether the item [`isequal`](@ref) to one of the elements. [`Dict`](@ref)s look for `key=>value` pairs, and the key is compared using [`isequal`](@ref). To test for the presence of a key in a dictionary, use [`haskey`](@ref) or `k in keys(dict)`. For these collections, the result is always a `Bool` and never `missing`.

# Examples

```jldoctest
julia> a = 1:3:20
1:3:19

julia> 4 in a
true

julia> 5 in a
false

julia> missing in [1, 2]
missing

julia> 1 in [2, missing]
missing

julia> 1 in [1, missing]
true

julia> missing in Set([1, 2])
false
```

---

```
in(p, sc)
```

Returns true if `p` is in `sc`. In the case that `sc` is a SortedDict or SortedMultiDict, `p` is a key=>value pair. In the case that `sc` is a SortedSet, `p` should be a key. Time: O(*c* log *n* + *d*) for SortedDict and SortedSet, where *d* stands for the time to compare two values. In the case of SortedMultiDict, the time is O(*c* log *n* + *dl*), and *l* stands for the number of entries that have the key of the given pair. (So therefore this call is inefficient if the same key addresses a large number of values, and an alternative should be considered.)

---

```
in(p, sc)
```

Returns true if `p` is in `sc`. In the case that `sc` is a SortedDict or SortedMultiDict, `p` is a key=>value pair. In the case that `sc` is a SortedSet, `p` should be a key. Time: O(*c* log *n* + *d*) for SortedDict and SortedSet, where *d* stands for the time to compare two values. In the case of SortedMultiDict, the time is O(*c* log *n* + *dl*), and *l* stands for the number of entries that have the key of the given pair. (So therefore this call is inefficient if the same key addresses a large number of values, and an alternative should be considered.)


In [55]:
function guardadictydatos(oldie::Dict, nuevos::Array, nombre::String)
    llaves=keys(oldie)
    
    if in( "dset", llaves)
        
        oldie["dset"]=nuevos
        println("guardando en $nombre")
        
        h5open(nombre, "w")  do file
            for k in llaves
                datos=oldie[k]
                write(file, "$k", datos)
            end
        end
        return
    else   
        error("Tu dict no trae dataset a remplazar")
    end
        
end

guardadictydatos (generic function with 1 method)

In [61]:
dirname(arxname), basename(arxname)

("/home/karel/BRWFiles/Isabel2019", "control_02.brw")

In [62]:
function cortayguarda(filenombre::String, fmem=0.25)
    

    dicth5 = brw_things( filenombre ); # variables útiles
    data = dicth5[ "dset" ]; # path al dataset (NO READ! to much)
    M=size(data, 1) # el tamaño de data
    chans=dicth5["Chs"]
    nch=size(chans, 1)
    nframes=Int(M/nch)
    
    println(nframes, "\n", M, "\n", M/4096)
    

    ncachos=min(ceil(Int, 1/fmem), 4) # al menos partela en 4
    fcacho=floor(Int, nframes/ncachos) # frames por cacho dara un poco menos que el total.
        
    paso=nch*fcacho #cuantos datos tiene cada cacho.
    
    for n = 1:ncachos  # numero de Β a cortar (1-> 4096,1->ω)
        
        datitos =reshape(data[((n-1)*paso+1):n*paso], nch, fcacho)
        
        nstring=lpad(n, 2, "0")
        dir=dirname(filenombre)
        fn=basename(filenombre)
        nuname=dir*"/Cacho_"*nstring*"_"*fn
        
        
        guardadictydatos(dicth5,datitos, nuname)
        
        println( "vamos en el cacho", n,  " de ", ncachos, " que va del cuadro ",
                (n-1)*fcacho+1, " al ",  n*fcacho, " guardado en $nuname")
    end
    print("bla")
    return  "tuabuela"
end

cortayguarda (generic function with 2 methods)

In [ ]:
if pmem < 0.25
    println("Tu archivo tiene tamaño razonable. Déjalo así.")
else
    a=brw_things(arxname)
    cortayguarda(arxname)
end

3213990
13164503040
3.21399e6
guardando en /home/karel/BRWFiles/Isabel2019/Cacho_01_control_02.brw
vamos en el cacho1 de 4 que va del cuadro 1 al 803497 guardado en /home/karel/BRWFiles/Isabel2019/Cacho_01_control_02.brw


In [33]:
function f(x)
    x=x+1
end

f (generic function with 1 method)

In [34]:
x=3
f(x)

4

In [35]:
x

3

In [6]:
typeof(a["dset"])

HDF5Dataset

In [7]:
arxdict=h5open(arxname)

HDF5 data file: /home/karel/BRWFiles/Isabel2019/control_02.brw

In [10]:
names(arxdict)

3-element Array{String,1}:
 "3BData"    
 "3BRecInfo" 
 "3BUserInfo"

In [14]:
names(arxdict["3BRecInfo/3BRecVars"])

7-element Array{String,1}:
 "BitDepth"       
 "ExperimentType" 
 "MaxVolt"        
 "MinVolt"        
 "NRecFrames"     
 "SamplingRate"   
 "SignalInversion"

In [66]:
b=read(arxdict["/3BData/RawEncodedTOC"])

3×1 Array{Int64,2}:
   89278
 3213990
       0

In [60]:
c=(arxdict["3BRecInfo/3BRecVars/NRecFrames"][1])[1]*4096

13530185728

In [63]:
(c-b)/4096

89278.0

In [69]:
b[1]/18

4959.888888888889

In [67]:
b[2]/b[1]

35.99979838258025

In [68]:
6*18

108